# Exploring Ensemble Methods
In this assignment, we will explore the use of boosting. We will use the pre-implemented gradient boosted trees in Scikit-learn. You will:

* Use Pandas to do some feature engineering.
* Train a boosted ensemble of decision-trees (gradient boosted trees) on the LendingClub dataset.
* Predict whether a loan will default along with prediction probabilities (on a validation set).
* Evaluate the trained model and compare it with a baseline.
* Find the most positive and negative loans using the learned model.
* Explore how the number of trees influences classification performance.


In [1]:
# Import some libs

import pandas
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Load LendingClub Dataset
This assignment will use the [LendingClub](https://www.lendingclub.com/) dataset used in the previous two assignments.

In [2]:
loans_df = pandas.read_csv('datasets/lending-club-data.csv', low_memory=False)

# safe_loans =  1 => safe
# safe_loans = -1 => risky
loans_df['safe_loans'] = loans_df['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans_df.drop(columns=['bad_loans'])

id  member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
0        1077501    1296599       5000         5000             4975   
1        1077430    1314167       2500         2500             2500   
2        1077175    1313524       2400         2400             2400   
3        1076863    1277178      10000        10000            10000   
4        1075269    1311441       5000         5000             5000   
5        1072053    1288686       3000         3000             3000   
6        1071795    1306957       5600         5600             5600   
7        1071570    1306721       5375         5375             5350   
8        1070078    1305201       6500         6500             6500   
9        1069908    1305008      12000        12000            12000   
10       1064687    1298717       9000         9000             9000   
11       1069866    1304956       3000         3000             3000   
12       1069057    1303503      10000        10000            10000   
13       1069759    1304871       1000         1000             1000   
14       1065775    1299699      10000        10000            10000   
15       1069971    1304884       3600         3600             3600   
16       1062474    1294539       6000         6000             6000   
17       1069742    1304855       9200         9200             9200   
18       1039153    1269083      21000        21000            21000   
19       1069710    1304821      10000        10000            10000   
20       1069700    1304810      10000        10000            10000   
21       1069559    1304634       6000         6000             6000   
22       1069697    1273773      15000        15000            15000   
23       1069800    1304679      15000        15000             8725   
24       1069657    1304764       5000         5000             5000   
25       1069799    1304678       4000         4000             4000   
26       1047704    1278806       8500         8500             8500   
27       1032111    1261745       4375         4375             4375   
28       1069539    1304608      31825        31825            31825   
29       1069591    1304289       5000         5000             5000   
...          ...        ...        ...          ...              ...   
122577  10086164   11938206       9000         9000             9000   
122578  10166062   12018153      15000        15000            15000   
122579  10106041   11958114      21000        21000            20800   
122580  10086141    1886905       7000         7000             6900   
122581  10115896   11967980       8250         8250             8250   
122582  10075730   11927830      10000        10000            10000   
122583  10075541   11927650       9450         9450             9450   
122584  10214568   12066719      12375        12375            12375   
122585   8944858   10736981      27575        27575            27575   
122586  10175470   12027558      18000        18000            18000   
122587   9654640   11506781      28000        28000            28000   
122588  10075202   11927320      14000        14000            14000   
122589  10105178   11957289      24000        24000            23800   
122590  10105115    9348848      10000        10000            10000   
122591   9924735   11776875      28000        28000            28000   
122592  10104967   11957097      10000        10000             9925   
122593  10064734   11916874      17000        17000            17000   
122594   9808485   11660306       8000         8000             8000   
122595   9001565   10792984      24000        24000            24000   
122596   9934676   11786815      35000        35000            35000   
122597   9828271   11680132      35000        35000            35000   
122598   9858164   11710004       5000         5000             5000   
122599   9767996   11619870      24000        24000            23925   
122600   7450630    1552101       6000         6000   

## Selecting features

In this assignment, we will be using a subset of features (categorical and numeric). The features we will be using are **described in the code comments** below. If you are a finance geek, the [LendingClub](https://www.lendingclub.com/) website has a lot more details about these features.

The features we will be using are described in the code comments below:

In [3]:
target = 'safe_loans'
features = ['grade',                     # grade of the loan (categorical)
            'sub_grade_num',             # sub-grade of the loan as a number from 0 to 1
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'payment_inc_ratio',         # ratio of the monthly payment to income
            'delinq_2yrs',               # number of delinquincies 
            'delinq_2yrs_zero',          # no delinquincies in last 2 years
            'inq_last_6mths',            # number of creditor inquiries in last 6 months
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'open_acc',                  # number of open credit accounts
            'pub_rec',                   # number of derogatory public records
            'pub_rec_zero',              # no derogatory public records
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
            'int_rate',                  # interest rate of the loan
            'total_rec_int',             # interest received to date
            'annual_inc',                # annual income of borrower
            'funded_amnt',               # amount committed to the loan
            'funded_amnt_inv',           # amount committed by investors for the loan
            'installment',               # monthly payment owed by the borrower
           ]

## Skipping observations with missing values

Recall from the lectures that one common approach to coping with missing values is to **skip** observations that contain missing values.

We run the following code to do so:

In [4]:
n = loans_df.shape[0]
loans_df = loans_df[features + [target]].dropna()
na = loans_df.shape[0]

print ("Drop {} and keep {}".format(n - na, na))

Drop 29 and keep 122578


## Subsample dataset to make sure classes are balanced
Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We used `seed = 1` so everyone gets the same results.

In [5]:
safe_loans_raw = loans_df[loans_df[target] == +1]
risky_loans_raw = loans_df[loans_df[target] == -1]

# Since there are fewer risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = risky_loans_raw.shape[0]/safe_loans_raw.shape[0]

risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(frac=percentage, random_state=1)

# Append the risky_loans with the downsampled version of safe_loans
loans_data = risky_loans.append(safe_loans)

print ("Percentage of safe loans                 : {}".format(safe_loans.shape[0] / loans_data.shape[0]))
print ("Percentage of risky loans                : {}".format(risky_loans.shape[0] / loans_data.shape[0]))
print ("Total number of loans in our new dataset : {}".format(loans_data.shape[0]))

Percentage of safe loans                 : 0.5
Percentage of risky loans                : 0.5
Total number of loans in our new dataset : 46294


## Transform categorical data into binary features

In [6]:
print(loans_data.dtypes)
categorical_variables = list(loans_data.select_dtypes(include=['object']).columns)
print(categorical_variables)

one_hot_data = pandas.get_dummies(loans_data[categorical_variables], prefix=categorical_variables)
# need to add inplace in oreder to drop columns.
loans_data.drop(columns=categorical_variables, axis=1, inplace=True)
loans_data = pandas.concat([loans_data, one_hot_data], axis=1)

print(loans_data['grade_A'].values.sum())
print (loans_data.dtypes)

grade                     object
sub_grade_num            float64
short_emp                  int64
emp_length_num             int64
home_ownership            object
dti                      float64
purpose                   object
payment_inc_ratio        float64
delinq_2yrs              float64
delinq_2yrs_zero         float64
inq_last_6mths           float64
last_delinq_none           int64
last_major_derog_none      int64
open_acc                 float64
pub_rec                  float64
pub_rec_zero             float64
revol_util               float64
total_rec_late_fee       float64
int_rate                 float64
total_rec_int            float64
annual_inc               float64
funded_amnt                int64
funded_amnt_inv            int64
installment              float64
safe_loans                 int64
dtype: object
['grade', 'home_ownership', 'purpose']
6522
sub_grade_num                 float64
short_emp                       int64
emp_length_num                  int64
dti

## Train-test split

We split the data into a train test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [7]:
np.random.seed(1)

train_data, validation_data = train_test_split(loans_data, test_size=0.2)

# Gradient boosted tree classifier
Gradient boosted trees are a powerful variant of boosting methods; they have been used to win many Kaggle competitions, and have been widely used in industry. We will explore the predictive power of multiple decision trees as opposed to a single decision tree.
<br>
Now, let's use the built-in scikit learn gradient boosting classifier [sklearn.ensemble.GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) to create a gradient boosted classifier on the training data. You will need to import **sklearn, sklearn.ensemble, and numpy.**
<br>
You will have to first convert the DataFrame into a numpy data matrix. See the API for more information. You will also have to extract the label column. Make sure to set **max_depth=6** and **n_estimators=5.**

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model_5 = GradientBoostingClassifier(n_estimators=5, max_depth=6, random_state=0)

X = train_data.loc[:, train_data.columns != target].values
y = train_data[target].values

model_5.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=5,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

# Making predictions

Just like we did in previous sections, let us consider a few positive and negative examples **from the validation set**. We will do the following:
* Predict whether or not a loan is likely to default.
* Predict the probability with which the loan is likely to default.

In [9]:
# Select all positive and negative examples.
validation_safe_loans = validation_data[validation_data[target] == 1]
validation_risky_loans = validation_data[validation_data[target] == -1]

# Select 2 examples from the validation set for positive & negative loans
sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

# Append the 4 examples into a single dataset
sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data[target]

93539     1
10285     1
121325   -1
74825    -1
Name: safe_loans, dtype: int64

### Predicting on sample validation data

For each row in the **sample_validation_data**, write code to make **model_5** predict whether or not the loan is classified as a **safe loan**.

**Hint:** Use the `predict` method in `model_5` for this.

In [10]:
print (model_5.predict(sample_validation_data.loc[:, train_data.columns != target].values))
print (model_5.predict_proba(sample_validation_data.loc[:, train_data.columns != target].values))

[-1  1 -1 -1]
[[0.54974932 0.45025068]
 [0.41588311 0.58411689]
 [0.6787601  0.3212399 ]
 [0.5564696  0.4435304 ]]


**Quiz Question:** What percentage of the predictions on `sample_validation_data` did `model_5` get correct?
<br>
**Your answer:**
<br>
**Checkpoint:** Can you verify that for all the predictions with `probability >= 0.5`, the model predicted the label **+1**?

## Evaluating the model on the validation data
Recall that the accuracy is defined as follows:
$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

Evaluate the accuracy of the **model_5** on the **validation_data**.

**Hint**: Use the `.score()` method in the model.

**Quiz Question:** What is the accuracy of **model_5**?
<br>
**Your answer:**

In [17]:
# YOUR CODE HERE
val_X = validation_data.loc[:, train_data.columns != target].values
val_y = validation_data[target].values
print (model_5.score(val_X, val_y))
predictions = model_5.predict(val_X)

0.6595744680851063


In [22]:
# YOUR CODE HERE
neg = val_y == -1
pos = val_y == 1

false_pos = np.sum(val_y[neg] != predictions[neg])
false_neg = np.sum(val_y[pos] != predictions[pos])

print (false_pos, false_neg)

1673 1479


**Quiz Question:** What is the **false positive**?
<br>
**Your answer:**
<br>
**Quiz Question:** What is the **false negative**?
<br>
**Your answer:**

## Comparison with decision trees

In the earlier assignment, we saw that the prediction accuracy of the decision trees was around **0.64** (rounded). In this assignment, we saw that **model_5** has an accuracy of **0.66** (rounded).

Here, we quantify the benefit of the extra 2% increase in accuracy of **model_5** in comparison with a single decision tree from the original decision tree assignment.

As we explored in the earlier assignment, we calculated the cost of the mistakes made by the model. We again consider the same costs as follows:

* **False negatives**: Assume a cost of \$10,000 per false negative.
* **False positives**: Assume a cost of \$20,000 per false positive.

Assume that the number of false positives and false negatives for the learned decision tree was

* **False negatives**: 1936
* **False positives**: 1503

Using the costs defined above and the number of false positives and false negatives for the decision tree, we can calculate the total cost of the mistakes made by the decision tree model as follows:

```
cost = $10,000 * 1936  + $20,000 * 1503 = $49,420,000
```

The total cost of the mistakes of the model is $49.42M. That is a **lot of money**!.

**Quiz Question**: Using the same costs of the false positives and false negatives, what is the cost of the mistakes made by the boosted tree model (**model_5**) as evaluated on the **validation_set**?
<br>
**Your answer:**

In [37]:
# YOUR CODE HERE
cost = 1e4*false_neg + 2e4*false_pos
print (cost)

48250000.0


## Most positive & negative loans.

In this section, we will find the loans that are most likely to be predicted **safe**. We can do this in a few steps:

* **Step 1**: Use the **model_5** (the model with 5 trees) and make **probability predictions** for all the loans in the **validation_data**.
* **Step 2**: Similar to what we did in the very first assignment, add the probability predictions as a column called **predictions** into the validation_data.
* **Step 3**: Sort the data (in descreasing order) by the probability predictions.

Start here with **Step 1** & **Step 2**. Make predictions using **model_5** for examples in the **validation_data**. Use `output_type = probability`.

In [43]:
# YOUR CODE HERE
validation_data['predictions'] = model_5.predict_proba(validation_data.loc[:, train_data.columns != target].values)[:,1]

/usr/local/anaconda3/envs/mlp-303x/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


**Checkpoint:** For each row, the probabilities should be a number in the range **[0, 1]**. We have provided a simple check here to make sure your answers are correct.

In [44]:
print (validation_data['predictions'][:5])

121325    0.321240
93539     0.450251
74825     0.443530
10285     0.584117
89414     0.551713
Name: predictions, dtype: float64


Now, we are ready to go to **Step 3**. You can now use the `prediction` column to sort the loans in **validation_data** (in descending order) by prediction probability. Find the top 5 loans with the highest probability of being predicted as a **safe loan**.

**Quiz Question**: What grades are the top 5 loans?
<br>
**Your answer**:

In [46]:
# YOUR CODE HERE
high_safe = validation_data.sort_values('predictions', ascending=False)
print (high_safe[:5].loc[:, ['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G', 'predictions', target]])

low_safe = validation_data.sort_values('predictions', ascending=True)
print (low_safe[:5].loc[:, ['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G', 'predictions', target]])

        grade_A  grade_B  grade_C  grade_D  grade_E  grade_F  grade_G  \
29080         1        0        0        0        0        0        0   
54108         1        0        0        0        0        0        0   
9624          1        0        0        0        0        0        0   
85067         1        0        0        0        0        0        0   
116768        1        0        0        0        0        0        0   

        predictions  safe_loans  
29080       0.66411           1  
54108       0.66411           1  
9624        0.66411           1  
85067       0.66411          -1  
116768      0.66411           1  
       grade_A  grade_B  grade_C  grade_D  grade_E  grade_F  grade_G  \
30677        1        0        0        0        0        0        0   
25697        0        0        1        0        0        0        0   
52678        0        1        0        0        0        0        0   
24272        0        1        0        0        0        0        0 

## Effect of adding more trees
In this assignment, we will train 5 different ensemble classifiers in the form of gradient boosted trees. We will train models with 10, 50, 100, 200, and 500 trees.  We use the **max_iterations** parameter in the boosted tree module. 

Let's get sarted with a model with **max_iterations = 10**:

In [47]:
model_10 = GradientBoostingClassifier(n_estimators=10, max_depth=6, random_state=0).fit(X, y)
model_50 = GradientBoostingClassifier(n_estimators=50, max_depth=6, random_state=0).fit(X, y)
model_100 = GradientBoostingClassifier(n_estimators=100, max_depth=6, random_state=0).fit(X, y)
model_200 = GradientBoostingClassifier(n_estimators=200, max_depth=6, random_state=0).fit(X, y)
model_500 = GradientBoostingClassifier(n_estimators=500, max_depth=6, random_state=0).fit(X, y)

In [48]:
print('Train accuraccy {}, val accuracy {}'.format(model_10.score(X, y), model_10.score(val_X, val_y)))
print('Train accuraccy {}, val accuracy {}'.format(model_50.score(X, y), model_50.score(val_X, val_y)))
print('Train accuraccy {}, val accuracy {}'.format(model_100.score(X, y), model_100.score(val_X, val_y)))
print('Train accuraccy {}, val accuracy {}'.format(model_200.score(X, y), model_200.score(val_X, val_y)))
print('Train accuraccy {}, val accuracy {}'.format(model_500.score(X, y), model_500.score(val_X, val_y)))

Train accuraccy 0.671769947347104, val accuracy 0.6619505346149692
Train accuraccy 0.7183745105980829, val accuracy 0.6784749972999244
Train accuraccy 0.7470500877548265, val accuracy 0.682903121287396
Train accuraccy 0.7874173079519373, val accuracy 0.6858192029376823
Train accuraccy 0.8665316592412583, val accuracy 0.6836591424559888


**Quiz Question:** Which model has the **best** accuracy on the **validation_data**?
<br>
**Your answer**:
<br>
**Quiz Question:** Is it always true that the model with the most trees will perform best on test data?
<br>
**Your answer**: